In [1]:
def improvement(row, e1, p1, e2, p2):
    m = 2
    # returns list of KIDs where 1 fits better than 2 with BIC
    dp = len(params.iloc[0][p2]) - len(params.iloc[0][p1])
    de1 = row[e1]
    de2 = row[e2] + m * dp * math.log(52)
    corr = de1 < de2
    if type(row) == pd.core.series.Series:
        return corr
    elif type(row) == pd.core.frame.DataFrame:
        return row.loc[corr]

In [4]:
def short_term():
    # TODO: contains 62% of all EBs, composed of 5% EBs
    
    # flat line fit
    # s = improvement(df, 'flatLineErrorL2', 'flatLineParam', 'straightLineErrorL2', 'straightLineParam')
        
    # high jitter
    s = df[df['Jitter'] > df['Average_Error']]
    
    cols = ['flatLineErrorL2', 'straightLineErrorL2', 'curveErrorL2', 'curtilErrorL2']
    
    # high error
    for col in cols:
        s = s.drop(s[s[col] > 20000].index)
    # s = s[s['curveErrorL2'] > rem['curveErrorL2'].median()]
    
    # > 0.5% flux variability
    s = s[s['stdev'] > 0.005]
    
    # scatter plot
    s = improvement(s, 'curveErrorL2', 'curveParamL2', 'straightLineErrorL2', 'straightLineParam')
    return s

In [ ]:
# increase penalty term of BIC, multiplier of klogn

In [3]:
def mid_term():
    s = improvement(df, 'curveErrorL2', 'curveParamL2', 'flatLineErrorL2', 'flatLineParam')
    s = improvement(s, 'curveErrorL2', 'curveParamL2', 'straightLineErrorL2', 'straightLineParam')
    # s = improvement(s, 'curtilErrorL2', 'curtilParamL2', 'flatLineErrorL2', 'flatLineParam')
    # s = improvement(s, 'curtilErrorL2', 'curtilParamL2', 'straightLineErrorL2', 'straightLineParam')
    s = improvement(s, 'curveErrorL2', 'curveParamL2', 'curtilErrorL2', 'curtilParamL2')
    
    # rem = df.drop(df[df['curtilErrorL2'] > 20000].index)
    
    # s = s[s['curveErrorL2'] < rem['curveErrorL2'].median()]
    # s = s[s['stdev'] / s['curtilErrorL2'] > rem['stdev'].median() / rem['curtilErrorL2'].median()]
    
    # > 0.1% flux variability
    s = s[s['stdev'] > 0.001]
    return s

In [1]:
def long_term():
    # straight line fit
    s = improvement(df, 'straightLineErrorL2', 'straightLineParam', 'flatLineErrorL2', 'flatLineParam')
    s = improvement(s, 'straightLineErrorL2', 'straightLineParam', 'curveErrorL2', 'curveParamL2')
    s = improvement(s, 'straightLineErrorL2', 'straightLineParam', 'curtilErrorL2', 'curtilParamL2')
    return s

In [5]:
def non_var():
    # flat line fit
    # s = improvement(df, 'flatLineErrorL2', 'flatLineParam', 'straightLineErrorL2', 'straightLineParam')
    # s = improvement(df, 'flatLineErrorL2', 'flatLineParam', 'curveErrorL2', 'curveParamL2')
    # s = improvement(s, 'flatLineErrorL2', 'flatLineParam', 'curtilErrorL2', 'curtilParamL2')
    
    # no jitter
    # s = s[s['Jitter'] == 0]
    
    s = df[~df['KIC'].isin(long_term()['KIC'])]
    s = s[~s['KIC'].isin(mid_term()['KIC'])]
    s = s[~s['KIC'].isin(short_term()['KIC'])]
    
    return s

In [11]:
def eb():
    repl = pd.DataFrame()
    # skip golden ffis, many errors/outliers
    for i in range(123 + 8, len(df.columns) - 1):
        add = df[df.iloc[:, i] < -3]
        repl = repl.append(add, ignore_index=True)
    return repl

In [6]:
def outl():
    count = [0] * len(cols)
    for i in range(123, len(df.columns) - 1):
        add = df[df.iloc[:, i] < -3]
        count[i - 123] += len(add)
    return count

In [8]:
def good():
    
    # systematics, star on edge of detector (drop one of four orientations)
    
    weird = pd.DataFrame()
        
    # big uncertainties
    for i in range(67, 119):
        add = df[df.iloc[:, i] > 0.1]
        weird = weird.append(add, ignore_index=True)
        
    err = df[df['curtilErrorL2'] > 20000]
    weird = weird.append(err, ignore_index=True)
    
    # huge var
    # weird = pd.merge(df[df['stdev'] > 5], weird, how='outer', on='KIC')
    
    # bad fits
        
    return df[~df['KIC'].isin(weird['KIC'])].reset_index(drop=True)